# Set up the environment

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import seaborn
from sklearn.manifold import TSNE, MDS

# Define a function to create a filename


In [ ]:
def getFileName(name, n_samples, model_name, layer_name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"   

# Get the model and layers' names and load the model RDM

In [ ]:
#define the required variables
multiple_models = 0 #1 is multiple models, 0 is not
dataset = 'imagenet'
arch = 'alexnet'
                                                ####################

ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/Scenes/'
layers_path = '/mnt/raid/ni/agnessa/RSA/'
if dataset == 'imagenet':
    NR_OF_SAMPLES = 10000
    path_dataset_specific = 'ImageNet/'
elif dataset == 'places365':
    NR_OF_SAMPLES = 10220
    path_dataset_specific = 'Places365/'
json_file_layers=os.path.join(layers_path + '/layer_names/',arch + '.json')

#for resnets, use the selected layers file
if 'res' in arch:
    json_file_layers=os.path.join(layers_path + '/layer_names/',arch + '_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)

#get the name of the model(s) and of the layers
num_layers_1 = 23
model_begin_1 = 1 #index of the first layer of the desired model
model_name_1 = selected_layers[model_begin_1].get('model') 
layer_names_1 = []

if multiple_models == 1:
    num_layers_2 = 16
    model_begin_2 = 16
    model_name_2 = selected_layers[model_begin_2].get('model')
    layer_names_2 = []

#append the layers 
for i in range(num_layers_1):
    layer_names_1.append(selected_layers[model_begin_1+i].get('layer'))   
if multiple_models == 1:
    for j in range(num_layers_2):
        layer_names_2.append(selected_layers[model_begin_2+j].get('layer'))
    
#Load the model RDM
layer_name_model_rdm = 'all'
if multiple_models == 1:
    model_name = model_name_1 + '_' + model_name_2
elif multiple_models == 0:
    model_name = model_name_1
path = os.path.join(ROOT_PATH + path_dataset_specific + 'Model_RDM/', getFileName("Model_RDM",NR_OF_SAMPLES,model_name,layer_name_model_rdm))
Model_RDM = np.load(path)
print('Loaded model ',model_name)

# Plot the Model RDM


In [ ]:
print("Plotting Model RDM for model -> ", model_name)
Model_RDM[np.isnan(Model_RDM)]=0.0 
fig = plt.figure(figsize=(18,15))
ax = seaborn.heatmap(Model_RDM, xticklabels = layer_names_1, yticklabels = layer_names_1, cmap='rainbow', vmin=0.0, vmax=1.0)
plt.xticks(rotation=90) 
plt.yticks(rotation=0)
plt.show()

# Save as a .png file

In [ ]:
path = os.path.join(ROOT_PATH + path_dataset_specific + 'Model_RDM_plots', getFileName("Model_RDM",NR_OF_SAMPLES,model_name,layer_name_model_rdm) + '.png')
fig.savefig(path)

# Multidimensional scaling

In [ ]:
mds = MDS(n_components=2, n_init=200, max_iter=2000, eps=0.0001, dissimilarity='precomputed')
data_embedded = mds.fit_transform(Model_RDM)

In [ ]:
sorted_layers = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
fig = plt.figure(figsize=(12,6)) 
plt.scatter(data_embedded[:, 0], data_embedded[:, 1], c=sorted_layers, cmap = 'rainbow', marker = '^')

for i in range(np.array(layer_names_1).shape[0]):
    plt.annotate(layer_names_1[i], (data_embedded[i, 0], data_embedded[i, 1]))

cbar = plt.colorbar()
# cbar.ax.set_ylabel('Layer', rotation=270)
plt.show()


In [ ]:
path = os.path.join(ROOT_PATH + path_dataset_specific +'Model_RDM_MDS', getFileName("Model_RDM_MDS_full",NR_OF_SAMPLES,model_name,layer_name_model_rdm) + '.png')
fig.savefig(path)